<a href="https://colab.research.google.com/github/stakunlena/ich_final_project/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка данных

## Загрузка библиотек и исходных данных

In [1]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np

# Загружаем исходные данные
from google.colab import drive # Импортируем библиотеку для работы с Google Drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
base_path = '/content/drive/MyDrive/P. Project 07.11/Data/'

# Загружаем файлы в датафреймы
df_calls = pd.read_excel(base_path + 'Calls (Done).xlsx')
df_contacts = pd.read_excel(base_path + 'Contacts (Done).xlsx')
df_deals = pd.read_excel(base_path + 'Deals (Done).xlsx')
df_spend = pd.read_excel(base_path + 'Spend (Done).xlsx')

# Определим словарь: имя переменной → объект DataFrame
# которым будем пользоваться для групповых операций с датафреймами
dfs = {
    'df_contacts': df_contacts,
    'df_calls': df_calls,
    'df_deals': df_deals,
    'df_spend': df_spend
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Кастомные функции

In [22]:
import re

def clean_columns_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует названия столбцов DataFrame в формат snake_case.

    В текущих датасетах выполняются следующие преобразования:
    1. Удаление лишних пробелов по краям.
    2. Замена пробелов и круглых скобок на символ подчёркивания.
    3. Приведение всех символов к нижнему регистру.

    Параметры
    ----------
    df : pandas.DataFrame
        Исходный DataFrame с оригинальными названиями столбцов.

    Возвращает
    ----------
    pandas.DataFrame
        Тот же объект DataFrame с обновлёнными названиями столбцов.
    """
    df.columns = (
        df.columns
        .map(lambda x: str(x).strip()) # обрезаем пробелы и защищаем от NaN
        .map(lambda x: re.sub(r'[\s()]+', '_', x)) # пробелы и скобки → "_"
        .str.lower() # всё в нижний регистр
    )
    return df

def normalize_id_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Приводит все столбцы, содержащие идентификаторы (id), к строковому типу
    и очищает значения от артефактов Excel и текстовых маркеров пропусков.

    Функция автоматически находит все колонки, кроме перечисленных в списке
    исключений, в названии которых присутствует
    подстрока 'id' (без учёта регистра), и выполняет следующие преобразования:
      1. приведение к типу pandas.StringDtype() — NA-aware строковый тип;
      2. удаление пробелов по краям строк;
      3. удаление суффиксов '.0', появляющихся после экспорта в Excel;
      4. замена строковых маркеров пропусков ('nan', 'NaN', '<NA>') на pd.NA.

    Параметры
    ----------
    df : pd.DataFrame
        Входной датафрейм с данными, содержащими столбцы идентификаторов.

    Возвращает
    ----------
    pd.DataFrame
        Датафрейм с обновлёнными столбцами идентификаторов, приведёнными
        к строковому типу и очищенными от артефактов.

    Пример
    -------
    >>> df = pd.DataFrame({
    ...     "Id": [5805028000018777278.0, 5805028000018777000.0, None],
    ...     "ContactID": ["5.805028E+18", "5805028000018777000", "<NA>"]
    ... })
    >>> normalize_id_columns(df)
                id          contactid
    0  5805028000018777278  5.805028E+18
    1  5805028000018777000  5805028000018777000
    2               <NA>              <NA>

    Примечание
    ----------
    Функция не удаляет пропуски и не восстанавливает значения.
    Её цель — унификация типа данных и формата хранения идентификаторов
    во всех таблицах проекта перед их объединением.
    """
    # Список исключений
    skip_cols = {"initial_amount_paid"}

    # Находим все колонки, содержащие подстроку "id", кроме тех, что в списке исключений
    id_cols = [c for c in df.columns if "id" in c.lower() and c.lower() not in skip_cols]

    # Приводим найденные колонки к единому типу и формату
    for col in id_cols:
        df[col] = (
            df[col]
            .astype("string")                             # безопасный NA-aware тип
            .str.strip()                                  # удаляем пробелы по краям
            .str.replace(r"\.0$", "", regex=True)          # убираем .0 после Excel
            .replace({"nan": pd.NA, "NaN": pd.NA, "<NA>": pd.NA})  # нормализуем пропуски
        )

    return df


import pandas as pd

def analyze_missing_ratio(df: pd.DataFrame, column: str, threshold: float = 0.01) -> None:
    """
    Анализирует долю пропусков в заданной колонке датафрейма и выводит
    рекомендации по дальнейшей обработке строк с пропущенными значениями.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для анализа.
    column : str
        Название столбца, в котором нужно проверить пропуски.
    threshold : float, optional
        Пороговое значение доли пропусков (по умолчанию 0.1 = 10%).
        Если доля пропусков меньше порога — рекомендуется удалить строки.
        Если доля выше порога — рекомендуется сохранить строки.

    Возвращает
    ----------
    None
        Функция только выводит информацию и не изменяет датафрейм.

    Пример
    -------
    >>> analyze_missing_ratio(df_calls, "contactid", threshold=0.1)
    Количество строк с пропусками в колонке 'contactid': 3933
    Доля пропусков от общего числа строк: 4.10%
    Рекомендация: доля пропусков меньше порогового значения (10.00%) → строки можно удалить.
    """
    # Проверка, что колонка существует
    if column not in df.columns:
        print(f"Ошибка: в датафрейме нет колонки '{column}'.")
        return

    # Подсчёт пропусков
    missing_rows = df[df[column].isna()]
    total_rows = len(df)
    missing_count = len(missing_rows)
    missing_share = missing_count / total_rows if total_rows > 0 else 0

    # Вывод статистики
    print(f"Количество строк с пропусками в колонке '{column}': {missing_count}")
    print(f"Доля пропусков от общего числа строк: {missing_share:.2%}")

    # Формирование рекомендации
    if missing_share < threshold:
        print(f"Рекомендация: доля пропусков меньше порогового значения ({threshold:.2%}) → строки можно удалить.")
    else:
        print(f"Рекомендация: доля пропусков превышает пороговое значение ({threshold:.2%}) → строки лучше сохранить.")

def drop_full_duplicates(df: pd.DataFrame, df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Проверяет и удаляет полные дубликаты строк в переданном датафрейме.

    Функция подсчитывает количество полностью совпадающих строк,
    выводит информацию о результатах проверки и возвращает очищенный датафрейм.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм, в котором нужно найти и удалить дубликаты.
    df_name : str, optional
        Название датафрейма (для удобного вывода в логах), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия исходного датафрейма без полных дублей, с обновлёнными индексами.
    """
    duplicates_count = df.duplicated().sum()
    print(f"[{df_name}] Найдено полных дублей: {duplicates_count}")

    if duplicates_count > 0:
        df = df.drop_duplicates().reset_index(drop=True)
        print(f"[{df_name}] Полные дубли удалены. Размер после очистки: {df.shape}")
    else:
        print(f"[{df_name}] Полных дублей не обнаружено. Размер датафрейма: {df.shape}")

    return df

def convert_datetime_columns(df: pd.DataFrame, date_columns: list[str], df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Преобразует указанные столбцы датафрейма в формат datetime64[ns].

    Функция безопасно приводит текстовые значения к типу datetime,
    используя формат '%d.%m.%Y %H:%M' (CRM-формат),
    и сообщает количество некорректных значений.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для преобразования.
    date_columns : list[str]
        Список названий столбцов, которые нужно преобразовать.
    df_name : str, optional
        Название датафрейма (для логирования), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия датафрейма с обновлёнными типами указанных столбцов.
    """
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format="%d.%m.%Y %H:%M", errors="coerce")
            invalid = df[col].isna().sum()
            print(f"[{df_name}] Столбец '{col}' преобразован в datetime64[ns]. Некорректных значений: {invalid}")
        else:
            print(f"[{df_name}] Столбец '{col}' не найден, пропускаем.")
    return df


## Ревью исходных данных. Определение количества пропусков

In [3]:
# Вывод информации по каждому датафрейму в цикле по элементам словаря dfs
for name, df in dfs.items():
    print(f"\n{name}")
    #print(f"  Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    #print(f"  Колонки: {df.columns.tolist()}")
    print(f"\n")
    print(f"  Информация о датасете:")
    print(df.info())
    print(f"\n")
    print(f"  Первые 5 строк датасета:")
    display(df.head())
    print(f"\n")
    print(f"  Количество пропусков в данных:")
    print(df.isna().sum())
    print(f"\n================================================")


df_contacts


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  int64 
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.8+ KB
None


  Первые 5 строк датасета:


,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,27.06.2023 11:28,22.12.2023 13:34
1,5805028000000872003,Charlie Davis,03.07.2023 11:31,21.05.2024 10:23
2,5805028000000889001,Bob Brown,02.07.2023 22:37,21.12.2023 13:17
3,5805028000000907006,Bob Brown,03.07.2023 05:44,29.12.2023 15:20
4,5805028000000939010,Nina Scott,04.07.2023 10:11,16.04.2024 16:14




  Количество пропусков в данных:
Id                    0
Contact Owner Name    0
Created Time          0
Modified Time         0
dtype: int64


df_calls


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                 

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,30.06.2023 08:43,John Doe,NaN,Inbound,171.0,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,30.06.2023 08:46,John Doe,NaN,Outbound,28.0,Attended Dialled,NaN,Completed,0.0,NaN
2,5805028000000764027,30.06.2023 08:59,John Doe,NaN,Outbound,24.0,Attended Dialled,NaN,Completed,0.0,NaN
3,5805028000000787003,30.06.2023 09:20,John Doe,5.805028e+18,Outbound,6.0,Attended Dialled,NaN,Completed,0.0,NaN
4,5805028000000768019,30.06.2023 09:30,John Doe,5.805028e+18,Outbound,11.0,Attended Dialled,NaN,Completed,0.0,NaN




  Количество пропусков в данных:
Id                                0
Call Start Time                   0
Call Owner Name                   0
CONTACTID                      3933
Call Type                         0
Call Duration (in seconds)       83
Call Status                       0
Dialled Number                95874
Outgoing Call Status           8999
Scheduled in CRM               8999
Tag                           95874
dtype: int64


df_deals


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21595 entries, 0 to 21594
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   21593 non-null  float64
 1   Deal Owner Name      21564 non-null  object 
 2   Closing Date         14645 non-null  object 
 3   Quality              19340 non-null  object 
 4   Stage                21593 non-null  object 
 5   Lost Reason          16124 non-null  object 
 6   Page

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5.805028e+18,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,...,NaN,NaN,21.06.2024 15:30,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
1,5.805028e+18,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,...,Web Developer,Morning,21.06.2024 15:23,6.0,NaN,0,2000,5.805028e+18,NaN,NaN
2,5.805028e+18,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,...,NaN,NaN,21.06.2024 14:45,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
3,5.805028e+18,Eva Kent,21.06.2024,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,01:00:04,bloggersvideo14com,...,NaN,NaN,21.06.2024 13:32,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN
4,5.805028e+18,Ben Hall,21.06.2024,D - Non Target,Lost,Non target,/eng,discovery_DE,00:53:12,website,...,NaN,NaN,21.06.2024 13:21,NaN,NaN,NaN,NaN,5.805028e+18,NaN,NaN




  Количество пропусков в данных:
Id                         2
Deal Owner Name           31
Closing Date            6950
Quality                 2255
Stage                      2
Lost Reason             5471
Page                       2
Campaign                5528
SLA                     6062
Content                 7448
Term                    9141
Source                     2
Payment Type           21099
Product                18003
Education Type         18295
Created Time               2
Course duration        18008
Months of study        20755
Initial Amount Paid    17430
Offer Total Amount     17410
Contact Name              63
City                   19084
Level of Deutsch       20344
dtype: int64


df_spend


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetim

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,NaN,NaN
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
3,2023-07-03,Google Ads,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN




  Количество пропусков в данных:
Date              0
Source            0
Campaign       5994
Impressions       0
Spend             0
Clicks            0
AdGroup        6828
Ad             6828
dtype: int64



## Приведение названия колонок всех датасетов к snake_case

In [4]:
# Изменение названий колонок в датафреймах с помощью функции clean_columns_names
for name, df in dfs.items():
    clean_columns_names(df)
    print(f"\n{name}")
    print(list(df.columns))


df_contacts
['id', 'contact_owner_name', 'created_time', 'modified_time']

df_calls
['id', 'call_start_time', 'call_owner_name', 'contactid', 'call_type', 'call_duration_in_seconds_', 'call_status', 'dialled_number', 'outgoing_call_status', 'scheduled_in_crm', 'tag']

df_deals
['id', 'deal_owner_name', 'closing_date', 'quality', 'stage', 'lost_reason', 'page', 'campaign', 'sla', 'content', 'term', 'source', 'payment_type', 'product', 'education_type', 'created_time', 'course_duration', 'months_of_study', 'initial_amount_paid', 'offer_total_amount', 'contact_name', 'city', 'level_of_deutsch']

df_spend
['date', 'source', 'campaign', 'impressions', 'spend', 'clicks', 'adgroup', 'ad']


## Обработка PK и FK таблиц

Проанализируем первичные и вторичные ключи исходных таблиц:


| Датафрейм | Ключ (PK/FK) | Тип PK/FK | Количество<br>пропусков | Планируемые действия |
|-|-|-|-|-|
| df_calls | id (PK)| int64 | 0 | Преобразовать в string |
| df_calls | contactid (FK)| float64 | 3933 | Преобразовать в string с сохранением пропусков |
| df_contacts | id (PK) | int64 | 0 | Преобразовать в string |
| df_deals | id (PK) | float64 | 2 | Удалить строки с пропусками, затем преобразовать в string |
| df_spend | — | — | — | Отсутствует. Добавить? |

Выводы:
* в датафрейме df_calls обнаружены 3933 строк с пропусками во вторичном ключе contactid — их слишком много, чтобы удалить данные строки и надо преобразовать поле в формат string с сохранением пропусков;
* в датафрейме df_deals обнаружены 2 строки с пропусками в первичном ключе, скорее всего это «мусор» в данных, перед преобразованием в тип string их надо проанализировать и удалить, если они не содержат значимой информации;
* в датафрейме df_spend специальных полей с идентификаторами нет, возможно понадобится впоследствии добавить в нее первичный ключ.

### Анализ и обработка пропусков в contactid (FK) датафрейма df_calls

In [5]:
analyze_missing_ratio(df_calls, "contactid")

Количество строк с пропусками в колонке 'contactid': 3933
Доля пропусков от общего числа строк: 4.10%
Рекомендация: доля пропусков превышает пороговое значение (1.00%) → строки лучше сохранить.


Выводы:
* доля строк с пропущенным contactid слишком велика, чтобы удалять их исходных данных;
* следует преобразовать это поле к типу strig с сохранением пропусков.

### Анализ и обработка пропусков в PK датафрейма df_deals

In [6]:
# Отфильтровать строки с пропусками в Id
missing_deals_id_rows = df_deals[df_deals['id'].isna()]

# Посмотреть количество, их долю и сами строки
analyze_missing_ratio(df_calls, "id")
#print(f"Количество строк в df_deals с пропущенным Id: {len(missing_deals_id_rows)}\n")
#print(f"Доля строк с пропущенным Id: {len(missing_deals_id_rows) / len(df_deals):.2%}\n")
print(f"Содержимое строк с пропущенным Id:\n\n{missing_deals_id_rows}")

Количество строк с пропусками в колонке 'id': 0
Доля пропусков от общего числа строк: 0.00%
Рекомендация: доля пропусков меньше порогового значения (1.00%) → строки можно удалить.
Содержимое строк с пропущенным Id:

       id deal_owner_name closing_date quality stage lost_reason page  \
21593 NaN             NaN          NaN     NaN   NaN         NaN  NaN   
21594 NaN             NaN          NaN     NaN   NaN         NaN  NaN   

      campaign  sla content  ... product education_type created_time  \
21593      NaN  NaN     NaN  ...     NaN            NaN          NaN   
21594      NaN  NaN     NaN  ...     NaN          #REF!          NaN   

      course_duration months_of_study initial_amount_paid  offer_total_amount  \
21593             NaN             NaN                 NaN                 NaN   
21594             NaN             NaN                 NaN                 NaN   

       contact_name city level_of_deutsch  
21593           NaN  NaN              NaN  
21594          

Выводы по результатам анализа пропусков в PK в df_deals:
* в таблице df_deals есть 2 строки, в которых одновременно пропущены все ключевые поля: Id, Stage, Page, Source, Created Time (и фактически не содержат ни одного полезного значени);
* индексы этих строк: 21593 и 21594;
* это типичные артефакты Excel-выгрузки — «пустые строки внизу файла»;
* эти строки следует удалить как технический мусор;
* строка 21594 содержит в колонке Education Type содержится значение #REF! — это помешает удалить её методом dropna и перед этим надо заменить текстовые «артефакты Excel» (#REF!, #N/A, #VALUE!, nan, и т.д.) на настоящие пропуски (pd.NA)

In [7]:
# Заменяем текстовые артефакты Excel на NaN
df_deals = df_deals.replace(['#REF!', '#N/A', '#VALUE!', 'NaN', 'nan', ''], np.nan)

# Удаляем строки с пропусками в PK из df_deals
df_deals = df_deals.dropna(how='all').reset_index(drop=True)

# Проверяем результат
print(df_deals.isna().sum())

id                         0
deal_owner_name           29
closing_date            6948
quality                 2253
stage                      0
lost_reason             5469
page                       0
campaign                5526
sla                     6060
content                 7446
term                    9139
source                     0
payment_type           21097
product                18001
education_type         18294
created_time               0
course_duration        18006
months_of_study        20753
initial_amount_paid    17428
offer_total_amount     17408
contact_name              61
city                   19082
level_of_deutsch       20342
dtype: int64


### Преобразование первичных ключей таблиц к типу string

In [8]:
# Преобразуем столбцы-ключи в таблицах к типу string
for name, df in dfs.items():
    df = normalize_id_columns(df)
    id_cols = [c for c in df.columns if "id" in c.lower()]
    print(name.lower())
    print(f"{df[id_cols].dtypes}\n")

# Проверяем, что в преобразованном столбце сохранились строки с пропусками в contactid
analyze_missing_ratio(df_calls, "contactid")

df_contacts
id    string[python]
dtype: object

df_calls
id           string[python]
contactid    string[python]
dtype: object

df_deals
id                     string[python]
initial_amount_paid            object
dtype: object

df_spend
Series([], dtype: object)

Количество строк с пропусками в колонке 'contactid': 3933
Доля пропусков от общего числа строк: 4.10%
Рекомендация: доля пропусков превышает пороговое значение (1.00%) → строки лучше сохранить.


## Предобработка df_contacts (таблица Contacts (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью кастомной функции drop_full_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся записи, каждая строка представляет уникальный контакт|
|2. Преобразование формата дат|Преобразовать столбцы created_time и modified_time к типу datetime64[ns] с указанием формата "%d.%m.%Y %H:%M"|Корректные типы данных, позволяющие выполнять сортировку и анализ по времени|
|3. Проверка логики дат|Проверить, что modified_time не раньше created_time, при нарушениях зафиксировать или удалить строки|Данные согласованы по времени, отсутствуют нелогичные даты|
|4. Очистка имён владельцев контактов|Удалить лишние пробелы, заменить двойные пробелы одним, привести к единому регистру методом .str.title()|Единый формат имён владельцев, готовых для дальнейшего сопоставления с другими таблицами|
|5. Проверка орфографических вариантов имён|Сравнить уникальные значения contact_owner_name, при необходимости создать словарь нормализации|Исправлены возможные опечатки и вариации написания, снижено количество уникальных значений|
|6. Финальная проверка типов данных|Убедиться, что id имеет строковый тип, contact_owner_name — строковый, даты — datetime|Все столбцы имеют корректные типы данных для анализа и объединений|


In [24]:
print(f"=== 1. Проверка и удаление полных дублей === \n")
# Применяем кастомную функцию для очистки дублей
df_contacts = drop_full_duplicates(df_contacts, "df_contacts")

print(f"\n=== 2. Преобразование формата дат === \n")
# Преобразование столбцов created_time и modified_time в формат datetime64[ns]
# c помощью кастомной функции convert_datetime_columns()
df_contacts = convert_datetime_columns(
    df_contacts,
    ["created_time", "modified_time"],
    df_name="df_contacts"
)


=== 1. Проверка и удаление полных дублей === 

[df_contacts] Найдено полных дублей: 0
[df_contacts] Полных дублей не обнаружено. Размер датафрейма: (18548, 4)

=== 2. Преобразование формата дат === 

[df_contacts] Столбец 'created_time' преобразован в datetime64[ns]. Некорректных значений: 0
[df_contacts] Столбец 'modified_time' преобразован в datetime64[ns]. Некорректных значений: 0
